In [1]:
from arcgis.gis import GIS
gis = GIS("pro")

In [2]:
import pandas as pd
from arcgis.features import SpatialDataFrame


In [3]:
def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

In [10]:
spa_con = r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WelshSPAMonitoringResults2017"
df = table_to_data_frame(spa_con)
df.head()

,Country,Site_Name,Site_Code,Site_Type,Grid_Ref,Feature_name__formal_,Feature_name__informal_,Reporting_Category,Season,Feature_Type,Reportin_1,Feature_condition__most_recent_assessment,Date_of_most_recent_assessment
OBJECTID,,,,,,,,,,,,,
1,EW,Bae Caerfyrddin/ Carmarthen Bay,UK9014091,SPA,SS280971,Melanitta nigra (Western Siberia/Western & Nor...,Common Scoter (W),Bird,W,Species,Birds - aggregations of non-breeding birds,NOT ASSESSED,N/A
2,W,Berwyn,UK9013111,SPA,SH919280,Milvus milvus (B),Red Kite (B),Bird,S,Species,Birds - aggregations of breeding birds,Unfavourable: Un-classified,36678
3,W,Berwyn,UK9013111,SPA,SH919280,Circus cyaneus (B),Hen Harrier (B),Bird,S,Species,Birds - aggregations of breeding birds,Favourable: Un-classified,38139
4,W,Berwyn,UK9013111,SPA,SH919280,Falco columbarius (B),Merlin (B),Bird,S,Species,Birds - aggregations of breeding birds,Unfavourable: Un-classified,38139
5,W,Berwyn,UK9013111,SPA,SH919280,Falco peregrinus (B),Peregrine (B),Bird,S,Species,Birds - aggregations of breeding birds,Unfavourable: Un-classified,37408


In [11]:
spa_features = df.groupby(["Site_Name","Site_Code"])["Feature_condition__most_recent_assessment"].count().reset_index(name='feat_count')
pd.set_option('display.max_rows', None)
spa_features

,Site_Name,Site_Code,feat_count
0,Bae Caerfyrddin/ Carmarthen Bay,UK9014091,1
1,Berwyn,UK9013111,4
2,Burry Inlet,UK9015011,13
3,Castlemartin Coast,UK9014061,1
4,Craig yr Aderyn (Bird`s Rock),UK9020283,2
5,Dyfi Estuary / Aber Dyfi,UK9020284,1
6,Elenydd – Mallaen,UK9014111,2
7,Glannau Aberdaron and Ynys Enlli/ Aberdaron Co...,UK9013121,3
8,Glannau Ynys Gybi/ Holy Island Coast,UK9013101,2
9,Grassholm,UK9014041,1


In [13]:
spa_fav = df.query('Feature_condition__most_recent_assessment.str.contains("Favourable")', engine='python')
spa_fav = spa_fav.groupby(["Site_Name","Site_Code"])["Feature_condition__most_recent_assessment"].count().reset_index(name='fav_count')
spa_fav

,Site_Name,Site_Code,fav_count
0,Berwyn,UK9013111,1
1,Burry Inlet,UK9015011,12
2,Castlemartin Coast,UK9014061,1
3,Dyfi Estuary / Aber Dyfi,UK9020284,1
4,Elenydd – Mallaen,UK9014111,2
5,Glannau Aberdaron and Ynys Enlli/ Aberdaron Co...,UK9013121,2
6,Glannau Ynys Gybi/ Holy Island Coast,UK9013101,1
7,Grassholm,UK9014041,1
8,Migneint–Arenig–Dduallt,UK9013131,1
9,"Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",UK9020282,1


In [14]:
spa_feature_summary = pd.merge(spa_features,spa_fav,on='Site_Name',how='outer' )
spa_feature_summary

,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count
0,Bae Caerfyrddin/ Carmarthen Bay,UK9014091,1,NaN,NaN
1,Berwyn,UK9013111,4,UK9013111,1.0
2,Burry Inlet,UK9015011,13,UK9015011,12.0
3,Castlemartin Coast,UK9014061,1,UK9014061,1.0
4,Craig yr Aderyn (Bird`s Rock),UK9020283,2,NaN,NaN
5,Dyfi Estuary / Aber Dyfi,UK9020284,1,UK9020284,1.0
6,Elenydd – Mallaen,UK9014111,2,UK9014111,2.0
7,Glannau Aberdaron and Ynys Enlli/ Aberdaron Co...,UK9013121,3,UK9013121,2.0
8,Glannau Ynys Gybi/ Holy Island Coast,UK9013101,2,UK9013101,1.0
9,Grassholm,UK9014041,1,UK9014041,1.0


In [15]:
spa_feature_summary['% Favourable'] = (spa_feature_summary['fav_count'] / spa_feature_summary['feat_count']*100)
#need to convert NaNs to 0
spa_feature_summary.fillna(0, inplace=True) 

spa_feature_summary

,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,Bae Caerfyrddin/ Carmarthen Bay,UK9014091,1,0,0.0,0.000000
1,Berwyn,UK9013111,4,UK9013111,1.0,25.000000
2,Burry Inlet,UK9015011,13,UK9015011,12.0,92.307692
3,Castlemartin Coast,UK9014061,1,UK9014061,1.0,100.000000
4,Craig yr Aderyn (Bird`s Rock),UK9020283,2,0,0.0,0.000000
5,Dyfi Estuary / Aber Dyfi,UK9020284,1,UK9020284,1.0,100.000000
6,Elenydd – Mallaen,UK9014111,2,UK9014111,2.0,100.000000
7,Glannau Aberdaron and Ynys Enlli/ Aberdaron Co...,UK9013121,3,UK9013121,2.0,66.666667
8,Glannau Ynys Gybi/ Holy Island Coast,UK9013101,2,UK9013101,1.0,50.000000
9,Grassholm,UK9014041,1,UK9014041,1.0,100.000000


### spa condition in AONBs ###

In [18]:
spa_aonb_50 = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SPA_AONB_Intersect50"
spa_aonb = SpatialDataFrame.from_featureclass(spa_aonb_50)
spa_aonb.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,AONB_NAME,AREA,PERCENTAGE,SHAPE
0,1,3,Burry Inlet,UK9015011,1992-07-14,6672.95,0.0,248306,196878,-4.194061,51.650173,2500,OS GB GRID,1,NC,4.0,2002-10-02,BJ MG,"-4,11,39","51,39,01",SS483968,Spherical,0,6650.59,1841.0,248306.379046,196878.316799,http://194.83.155.90/olibcgi?infile=details.gl...,,4,Burry Inlet,GOWER,41.713564,62.511388,"{'rings': [[[257821.09999999963, 196920.800000..."
1,2,8,Glannau Ynys Gybi / Holy Island Coast,UK9013101,2002-05-29,609.36,0.0,220956,381673,-4.688333,53.301667,2500,OS GB GRID,1,NC,8.0,2002-10-03,WR MG,"-4,41,22","53,18,08",SH230804,Spherical,1,606.97,1846.0,223066.234277,380431.221423,http://194.83.155.90/olibcgi?infile=details.gl...,,6,Glannau Ynys Gybi / Holy Island Coast,YNYS MON/ANGLESEY,5.936838,97.426867,"{'rings': [[[223615.13999999966, 383870.050000..."
2,3,10,"Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",UK9020282,2003-06-10,372.95,0.0,230670,324743,-4.512778,52.793611,2500,OS GB GRID,1,NC,6.0,2003-05-19,SM CH MG,"-4,30,46","52,47,37",SH303247,Cartesian,1,371.55,2793.0,230335.066125,324766.095671,http://194.83.155.90/olibcgi?infile=details.gl...,,8,"Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",LLYN,3.727521,99.947905,"{'rings': [[[228625.90000000037, 324056.199999..."
3,4,17,Ynys Seiriol / Puffin Island,UK9020285,2002-02-01,31.33,0.0,265090,381948,-4.026667,53.317222,2500,OS GB GRID,1,C,5.0,2000-11-07,SM,"-4,01,36","53,19,02",SH650820,Spherical,1,31.21,2639.0,265041.490025,382064.489527,http://194.83.155.90/olibcgi?infile=details.gl...,,9,Ynys Seiriol / Puffin Island,YNYS MON/ANGLESEY,0.313270,99.999852,"{'rings': [[[264584.53000000026, 381393.810000..."


In [16]:
#spa_feature_summary.describe()
cols = spa_feature_summary.columns.tolist()
cols
#spa_aonb.columns[2] = spa_aonb.columns[2].astype(str)
#spa_feature_summary['Site_Name'] = spa_feature_summary['Site_Name'].str.strip()

spa_feature_summary

,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,Bae Caerfyrddin/ Carmarthen Bay,UK9014091,1,0,0.0,0.000000
1,Berwyn,UK9013111,4,UK9013111,1.0,25.000000
2,Burry Inlet,UK9015011,13,UK9015011,12.0,92.307692
3,Castlemartin Coast,UK9014061,1,UK9014061,1.0,100.000000
4,Craig yr Aderyn (Bird`s Rock),UK9020283,2,0,0.0,0.000000
5,Dyfi Estuary / Aber Dyfi,UK9020284,1,UK9020284,1.0,100.000000
6,Elenydd – Mallaen,UK9014111,2,UK9014111,2.0,100.000000
7,Glannau Aberdaron and Ynys Enlli/ Aberdaron Co...,UK9013121,3,UK9013121,2.0,66.666667
8,Glannau Ynys Gybi/ Holy Island Coast,UK9013101,2,UK9013101,1.0,50.000000
9,Grassholm,UK9014041,1,UK9014041,1.0,100.000000


In [24]:
# needs an outer join here but caused an error, try and resolve also required elsewhere to brign back sites with no features at all
spa_aonb_con_merge = pd.merge(spa_aonb,spa_feature_summary,left_on='SPA_code', right_on='Site_Code_x')
spa_aonb_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,AONB_NAME,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,1,3,Burry Inlet,UK9015011,1992-07-14,6672.95,0.0,248306,196878,-4.194061,51.650173,2500,OS GB GRID,1,NC,4.0,2002-10-02,BJ MG,"-4,11,39","51,39,01",SS483968,Spherical,0,6650.59,1841.0,248306.379046,196878.316799,http://194.83.155.90/olibcgi?infile=details.gl...,,4,Burry Inlet,GOWER,41.713564,62.511388,"{'rings': [[[257821.09999999963, 196920.800000...",Burry Inlet,UK9015011,13,UK9015011,12.0,92.307692
1,2,8,Glannau Ynys Gybi / Holy Island Coast,UK9013101,2002-05-29,609.36,0.0,220956,381673,-4.688333,53.301667,2500,OS GB GRID,1,NC,8.0,2002-10-03,WR MG,"-4,41,22","53,18,08",SH230804,Spherical,1,606.97,1846.0,223066.234277,380431.221423,http://194.83.155.90/olibcgi?infile=details.gl...,,6,Glannau Ynys Gybi / Holy Island Coast,YNYS MON/ANGLESEY,5.936838,97.426867,"{'rings': [[[223615.13999999966, 383870.050000...",Glannau Ynys Gybi/ Holy Island Coast,UK9013101,2,UK9013101,1.0,50.000000
2,3,10,"Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",UK9020282,2003-06-10,372.95,0.0,230670,324743,-4.512778,52.793611,2500,OS GB GRID,1,NC,6.0,2003-05-19,SM CH MG,"-4,30,46","52,47,37",SH303247,Cartesian,1,371.55,2793.0,230335.066125,324766.095671,http://194.83.155.90/olibcgi?infile=details.gl...,,8,"Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",LLYN,3.727521,99.947905,"{'rings': [[[228625.90000000037, 324056.199999...","Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...",UK9020282,2,UK9020282,1.0,50.000000
3,4,17,Ynys Seiriol / Puffin Island,UK9020285,2002-02-01,31.33,0.0,265090,381948,-4.026667,53.317222,2500,OS GB GRID,1,C,5.0,2000-11-07,SM,"-4,01,36","53,19,02",SH650820,Spherical,1,31.21,2639.0,265041.490025,382064.489527,http://194.83.155.90/olibcgi?infile=details.gl...,,9,Ynys Seiriol / Puffin Island,YNYS MON/ANGLESEY,0.313270,99.999852,"{'rings': [[[264584.53000000026, 381393.810000...",Ynys Seiriol / Puffin Island,UK9020285,1,UK9020285,1.0,100.000000


In [25]:
spa_aonb_final = spa_aonb_con_merge.groupby("AONB_NAME", as_index=False)["feat_count","fav_count"].sum()
spa_aonb_final['% Favourable'] = (spa_aonb_final['fav_count'] / spa_aonb_final['feat_count']*100)
spa_aonb_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,AONB_NAME,feat_count,fav_count,% Favourable
0,GOWER,13,12.0,92.307692
1,LLYN,2,1.0,50.000000
2,YNYS MON/ANGLESEY,3,2.0,66.666667


### SPA condition in National Parks ###

In [13]:
spa_np_50 = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SPA_NP_Intersect50"
spa_np = SpatialDataFrame.from_featureclass(spa_np_50)
spa_np.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,NP_NAME,AREA,PERCENTAGE,SHAPE
0,1,4,Castlemartin Coast,UK9014061,1996-01-12,1119.03,0.0,192184,196428,-5.003971,51.628187,2500,OS GB GRID,1,NC,4.0,2001-05-14,ABM,"-5,0,14","51,37,41",SR921964,Spherical,0,1114.74,1842.0,192183.767268,196427.785325,http://194.83.155.90/olibcgi?infile=details.gl...,,3,Castlemartin Coast,Pembrokeshire Coast,11.151128,99.649553,"{'rings': [[[187683.4620000003, 200624.4570000..."
1,2,5,Craig yr Aderyn (Bird's Rock),UK9020283,2001-07-16,89.43,0.0,264613,306729,-4.002778,52.641944,2500,OS GB GRID,1,C,2.0,2000-11-17,SM,"-4,00,10","52,38,31",SH646067,Spherical,1,89.12,2610.0,264613.247426,306728.748101,http://194.83.155.90/olibcgi?infile=details.gl...,,4,Craig yr Aderyn (Bird's Rock),Snowdonia,0.894330,100.000000,"{'rings': [[[264910.8959999997, 306738.2310000..."
2,3,9,Migneint-Arenig-Dduallt,UK9013131,2003-04-03,19968.32,0.0,280430,343695,-3.782200,52.977222,2500,OS GB GRID,1,NC,3.0,2003-03-17,SDV,"-3,46,56","52,58,38",SH810394,Cartesian,1,19899.91,2714.0,281016.527161,339459.715020,http://194.83.155.90/olibcgi?infile=details.gl...,,7,Migneint-Arenig-Dduallt,Snowdonia,196.615012,98.463490,"{'rings': [[[270677.28000000026, 339886.609999..."
3,4,11,Ramsey and St David's Peninsula Coast,UK9014062,1996-07-24,846.24,0.0,173832,225399,-5.287526,51.881163,2500,OS GB GRID,1,NC,4.0,2001-05-30,ABM / MG,"-5,17,15","51,52,52",SM738253,Spherical,0,842.78,1853.0,173831.736419,225399.129754,http://194.83.155.90/olibcgi?infile=details.gl...,,8,Ramsey and St David's Peninsula Coast,Pembrokeshire Coast,8.298460,98.062804,"{'rings': [[[172389.8200000003, 228174.6999999..."


In [14]:
spa_np['SPA_Name'] = spa_np['SPA_Name'].str.strip()
spa_np.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,NP_NAME,AREA,PERCENTAGE,SHAPE
0,1,4,Castlemartin Coast,UK9014061,1996-01-12,1119.03,0.0,192184,196428,-5.003971,51.628187,2500,OS GB GRID,1,NC,4.0,2001-05-14,ABM,"-5,0,14","51,37,41",SR921964,Spherical,0,1114.74,1842.0,192183.767268,196427.785325,http://194.83.155.90/olibcgi?infile=details.gl...,,3,Castlemartin Coast,Pembrokeshire Coast,11.151128,99.649553,"{'rings': [[[187683.4620000003, 200624.4570000..."
1,2,5,Craig yr Aderyn (Bird's Rock),UK9020283,2001-07-16,89.43,0.0,264613,306729,-4.002778,52.641944,2500,OS GB GRID,1,C,2.0,2000-11-17,SM,"-4,00,10","52,38,31",SH646067,Spherical,1,89.12,2610.0,264613.247426,306728.748101,http://194.83.155.90/olibcgi?infile=details.gl...,,4,Craig yr Aderyn (Bird's Rock),Snowdonia,0.894330,100.000000,"{'rings': [[[264910.8959999997, 306738.2310000..."
2,3,9,Migneint-Arenig-Dduallt,UK9013131,2003-04-03,19968.32,0.0,280430,343695,-3.782200,52.977222,2500,OS GB GRID,1,NC,3.0,2003-03-17,SDV,"-3,46,56","52,58,38",SH810394,Cartesian,1,19899.91,2714.0,281016.527161,339459.715020,http://194.83.155.90/olibcgi?infile=details.gl...,,7,Migneint-Arenig-Dduallt,Snowdonia,196.615012,98.463490,"{'rings': [[[270677.28000000026, 339886.609999..."
3,4,11,Ramsey and St David's Peninsula Coast,UK9014062,1996-07-24,846.24,0.0,173832,225399,-5.287526,51.881163,2500,OS GB GRID,1,NC,4.0,2001-05-30,ABM / MG,"-5,17,15","51,52,52",SM738253,Spherical,0,842.78,1853.0,173831.736419,225399.129754,http://194.83.155.90/olibcgi?infile=details.gl...,,8,Ramsey and St David's Peninsula Coast,Pembrokeshire Coast,8.298460,98.062804,"{'rings': [[[172389.8200000003, 228174.6999999..."


### merge on site code due to string unicode issues with names

In [15]:
spa_np_count_merge = pd.merge(spa_np,spa_feature_summary,left_on='SPA_code', right_on='Site_Code_x')
spa_np_count_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,NP_NAME,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,1,4,Castlemartin Coast,UK9014061,1996-01-12,1119.03,0.0,192184,196428,-5.003971,51.628187,2500,OS GB GRID,1,NC,4.0,2001-05-14,ABM,"-5,0,14","51,37,41",SR921964,Spherical,0,1114.74,1842.0,192183.767268,196427.785325,http://194.83.155.90/olibcgi?infile=details.gl...,,3,Castlemartin Coast,Pembrokeshire Coast,11.151128,99.649553,"{'rings': [[[187683.4620000003, 200624.4570000...",Castlemartin Coast,UK9014061,1,UK9014061,1.0,100.000000
1,2,5,Craig yr Aderyn (Bird's Rock),UK9020283,2001-07-16,89.43,0.0,264613,306729,-4.002778,52.641944,2500,OS GB GRID,1,C,2.0,2000-11-17,SM,"-4,00,10","52,38,31",SH646067,Spherical,1,89.12,2610.0,264613.247426,306728.748101,http://194.83.155.90/olibcgi?infile=details.gl...,,4,Craig yr Aderyn (Bird's Rock),Snowdonia,0.894330,100.000000,"{'rings': [[[264910.8959999997, 306738.2310000...",Craig yr Aderyn (Bird`s Rock),UK9020283,2,0,0.0,0.000000
2,3,9,Migneint-Arenig-Dduallt,UK9013131,2003-04-03,19968.32,0.0,280430,343695,-3.782200,52.977222,2500,OS GB GRID,1,NC,3.0,2003-03-17,SDV,"-3,46,56","52,58,38",SH810394,Cartesian,1,19899.91,2714.0,281016.527161,339459.715020,http://194.83.155.90/olibcgi?infile=details.gl...,,7,Migneint-Arenig-Dduallt,Snowdonia,196.615012,98.463490,"{'rings': [[[270677.28000000026, 339886.609999...",Migneint–Arenig–Dduallt,UK9013131,3,UK9013131,1.0,33.333333
3,4,11,Ramsey and St David's Peninsula Coast,UK9014062,1996-07-24,846.24,0.0,173832,225399,-5.287526,51.881163,2500,OS GB GRID,1,NC,4.0,2001-05-30,ABM / MG,"-5,17,15","51,52,52",SM738253,Spherical,0,842.78,1853.0,173831.736419,225399.129754,http://194.83.155.90/olibcgi?infile=details.gl...,,8,Ramsey and St David's Peninsula Coast,Pembrokeshire Coast,8.298460,98.062804,"{'rings': [[[172389.8200000003, 228174.6999999...",Ramsey and St David`s Peninsula Coast,UK9014062,1,UK9014062,1.0,100.000000


In [16]:
spa_np_final = spa_np_count_merge.groupby("NP_NAME", as_index=False)["feat_count","fav_count"].sum()
spa_np_final['% Favourable'] = (spa_np_final['fav_count'] / spa_np_final['feat_count']*100)
spa_np_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,NP_NAME,feat_count,fav_count,% Favourable
0,Pembrokeshire Coast,2,2.0,100.0
1,Snowdonia,5,1.0,20.0


## SAC Condition ##

In [4]:
sac_con = r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WelshSACMonitoringResults2017"
sacdf = table_to_data_frame(sac_con)
sacdf.head()

,Country,Site_Name,Site_Code,Site_Type,Grid_Ref,Feature_name__formal_,Feature_name__informal_,Broad_environment,Feature_Type,Feature_category,Feature_condition__Most_recent_assessment,Date_of_most_recent_assessment
OBJECTID,,,,,,,,,,,,
1,EW,Aberbargoed Grasslands,UK0030071,SAC,ST163992,"Euphydryas (Eurodryas, Hypodryas) aurinia (R)",Marsh fritillary butterfly. (R),Terrestrial,Species,Butterflies,Unfavourable: Un-classified,39934
2,EW,Aberbargoed Grasslands,UK0030071,SAC,ST163992,"Molinia meadows on calcareous, peaty or clayey...",Purple moor-grass meadows.,Terrestrial,Habitat,"Fen, marsh and swamp",Unfavourable: Un-classified,41791
3,EW,Afon Eden – Cors Goch Trawsfynydd,UK0030075,SAC,SH720271,Active raised bogs,Active raised bogs.,Terrestrial,Habitat,Bogs,Unfavourable: No-change,42248
4,EW,Afon Eden – Cors Goch Trawsfynydd,UK0030075,SAC,SH720271,Luronium natans (R),Floating water-plantain. (R),Freshwater,Species,Vascular Plants,Favourable: Maintained,40299
5,EW,Afon Eden – Cors Goch Trawsfynydd,UK0030075,SAC,SH720271,Lutra lutra (R),Otter. (R),Freshwater,Species,Mammals,Unfavourable: Un-classified,40238


In [5]:
sac_features = sacdf.groupby(["Site_Name","Site_Code"])["Feature_condition__Most_recent_assessment"].count().reset_index(name='feat_count')
pd.set_option('display.max_rows', None)
sac_features.head()

,Site_Name,Site_Code,feat_count
0,Aberbargoed Grasslands,UK0030071,2
1,Afon Eden – Cors Goch Trawsfynydd,UK0030075,5
2,Afon Gwyrfai a Llyn Cwellyn,UK0030046,5
3,Afon Teifi/ River Teifi,UK0012670,9
4,Afon Tywi/ River Tywi,UK0013010,7


In [6]:
sac_fav = sacdf.query('Feature_condition__Most_recent_assessment.str.contains("Favourable")', engine='python')
sac_fav = sac_fav.groupby(["Site_Name","Site_Code"])["Feature_condition__Most_recent_assessment"].count().reset_index(name='fav_count')
sac_fav.head()

,Site_Name,Site_Code,fav_count
0,Afon Eden – Cors Goch Trawsfynydd,UK0030075,1
1,Afon Gwyrfai a Llyn Cwellyn,UK0030046,3
2,Afon Teifi/ River Teifi,UK0012670,6
3,Afon Tywi/ River Tywi,UK0013010,1
4,Afonydd Cleddau/ Cleddau Rivers,UK0030074,1


In [7]:
sac_feature_summary = pd.merge(sac_features,sac_fav,on='Site_Name', how='outer')
sac_feature_summary.head()

,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count
0,Aberbargoed Grasslands,UK0030071,2,NaN,NaN
1,Afon Eden – Cors Goch Trawsfynydd,UK0030075,5,UK0030075,1.0
2,Afon Gwyrfai a Llyn Cwellyn,UK0030046,5,UK0030046,3.0
3,Afon Teifi/ River Teifi,UK0012670,9,UK0012670,6.0
4,Afon Tywi/ River Tywi,UK0013010,7,UK0013010,1.0


In [9]:
sac_feature_summary['% Favourable'] = (sac_feature_summary['fav_count'] / sac_feature_summary['feat_count']*100)
#need to convert NaNs to 0
sac_feature_summary.fillna(0, inplace=True) 

sac_feature_summary

,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,Aberbargoed Grasslands,UK0030071,2,0,0.0,0.000000
1,Afon Eden – Cors Goch Trawsfynydd,UK0030075,5,UK0030075,1.0,20.000000
2,Afon Gwyrfai a Llyn Cwellyn,UK0030046,5,UK0030046,3.0,60.000000
3,Afon Teifi/ River Teifi,UK0012670,9,UK0012670,6.0,66.666667
4,Afon Tywi/ River Tywi,UK0013010,7,UK0013010,1.0,14.285714
5,Afonydd Cleddau/ Cleddau Rivers,UK0030074,8,UK0030074,1.0,12.500000
6,Alyn Valley Woods/ Coedwigoedd Dyffryn Alun,UK0030078,3,0,0.0,0.000000
7,Bae Cemlyn/ Cemlyn Bay,UK0030114,2,0,0.0,0.000000
8,Berwyn a Mynyddoedd de Clwyd/ Berwyn and South...,UK0012926,6,UK0012926,3.0,50.000000
9,Blackmill Woodlands,UK0030090,1,0,0.0,0.000000


### SAC Condition in AONBs ###

In [22]:
sac_aonb_50 = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SAC_AONB_Intersect50"
sac_aonb = SpatialDataFrame.from_featureclass(sac_aonb_50)
sac_aonb.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,AONB_NAME,AREA,PERCENTAGE,SHAPE
0,1,1,Y Twyni o Abermenai i Aberffraw / Abermenai to...,UK0020021,Designated,p,2004-12-13,1870.929250,0.0,241345.0,364288.0,-4.374071,53.152017,2500,OS GB GRID,1,,4.0,22,2004-07-30,"BJ, MG","-4,22,27","53,9,7",SH413642,Cartesian,1870.929250,1806.0,http://194.83.155.90/olibcgi?infile=details.gl...,,27,Y Twyni o Abermenai i Aberffraw / Abermenai to...,YNYS MON/ANGLESEY,16.260706,86.912460,"{'rings': [[[238352.5, 365736.5], [238425.5, 3..."
1,2,4,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,UK0012727,Designated,n,2002-10-25,389.047400,0.0,353057.0,195727.0,-2.680048,51.658310,2500,OS GB GRID,1,C,7.0,22,2005-07-11,EN/MG,"-2,40,48","51,39,30",ST530957,Cartesian,389.047400,1829.0,http://194.83.155.90/olibcgi?infile=details.gl...,,24,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,WYE VALLEY (England),0.000084,0.002150,"{'rings': [[[353325.22300000023, 197365.512000..."
2,3,5,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,UK0012727,Designated,n,2004-12-13,526.290681,0.0,354621.0,215272.0,-2.660010,51.834200,2500,OS GB GRID,1,,6.0,22,2002-08-12,BJ,"-2,39,36","51,50,3",S0546152,Cartesian,526.290681,1829.0,http://194.83.155.90/olibcgi?infile=details.gl...,,23,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,WYE VALLEY (England),5.262720,99.996444,"{'rings': [[[354320.88100000005, 215390.314999..."
3,4,6,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2005-04-01,1.301962,0.0,352203.0,210725.0,-2.694459,51.793082,2500,OS GB GRID,1,C,6.0,23,2005-07-18,EN,"-2,41,40","51,47,35",SO522107,Cartesian,1.301962,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,22,Wye Valley and Forest of Dean Bat Sites / Safl...,WYE VALLEY (Wales),0.007473,57.400803,"{'rings': [[[352181.9400000004, 210683.1799999..."
4,5,45,Gower Commons / Tiroedd Comin Gwyr,UK0012685,Designated,n,2004-12-13,1776.310557,0.0,249789.0,190026.0,-4.169754,51.588965,2500,OS GB GRID,1,NC,6.0,25,2003-02-28,WR/MG,"-4,10,11","51,35,20",SS497900,Cartesian,1776.310557,1808.0,http://194.83.155.90/olibcgi?infile=details.gl...,,12,Gower Commons / Tiroedd Comin Gwyr,GOWER,17.262288,97.180575,"{'rings': [[[251021.5, 190443.90000000037], [2..."


In [23]:
sac_aonb_con_merge = pd.merge(sac_aonb,sac_feature_summary,left_on='Site_code', right_on='Site_Code_x')
sac_aonb_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,AONB_NAME,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,1,1,Y Twyni o Abermenai i Aberffraw / Abermenai to...,UK0020021,Designated,p,2004-12-13,1870.929250,0.0,241345.0,364288.0,-4.374071,53.152017,2500,OS GB GRID,1,,4.0,22,2004-07-30,"BJ, MG","-4,22,27","53,9,7",SH413642,Cartesian,1870.929250,1806.0,http://194.83.155.90/olibcgi?infile=details.gl...,,27,Y Twyni o Abermenai i Aberffraw / Abermenai to...,YNYS MON/ANGLESEY,16.260706,86.912460,"{'rings': [[[238352.5, 365736.5], [238425.5, 3...",Y Twyni o Abermenai i Aberffraw/ Abermenai to ...,UK0020021,8,0,0.0,0.0
1,2,4,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,UK0012727,Designated,n,2002-10-25,389.047400,0.0,353057.0,195727.0,-2.680048,51.658310,2500,OS GB GRID,1,C,7.0,22,2005-07-11,EN/MG,"-2,40,48","51,39,30",ST530957,Cartesian,389.047400,1829.0,http://194.83.155.90/olibcgi?infile=details.gl...,,24,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,WYE VALLEY (England),0.000084,0.002150,"{'rings': [[[353325.22300000023, 197365.512000...",Wye Valley Woodlands/ Coetiroedd Dyffryn Gwy,UK0012727,4,UK0012727,2.0,50.0
2,3,5,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,UK0012727,Designated,n,2004-12-13,526.290681,0.0,354621.0,215272.0,-2.660010,51.834200,2500,OS GB GRID,1,,6.0,22,2002-08-12,BJ,"-2,39,36","51,50,3",S0546152,Cartesian,526.290681,1829.0,http://194.83.155.90/olibcgi?infile=details.gl...,,23,Wye Valley Woodlands / Coetiroedd Dyffryn Gwy ...,WYE VALLEY (England),5.262720,99.996444,"{'rings': [[[354320.88100000005, 215390.314999...",Wye Valley Woodlands/ Coetiroedd Dyffryn Gwy,UK0012727,4,UK0012727,2.0,50.0
3,4,6,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2005-04-01,1.301962,0.0,352203.0,210725.0,-2.694459,51.793082,2500,OS GB GRID,1,C,6.0,23,2005-07-18,EN,"-2,41,40","51,47,35",SO522107,Cartesian,1.301962,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,22,Wye Valley and Forest of Dean Bat Sites / Safl...,WYE VALLEY (Wales),0.007473,57.400803,"{'rings': [[[352181.9400000004, 210683.1799999...",Wye Valley and Forest of Dean Bat Sites/ Safle...,UK0014794,2,UK0014794,2.0,100.0
4,5,45,Gower Commons / Tiroedd Comin Gwyr,UK0012685,Designated,n,2004-12-13,1776.310557,0.0,249789.0,190026.0,-4.169754,51.588965,2500,OS GB GRID,1,NC,6.0,25,2003-02-28,WR/MG,"-4,10,11","51,35,20",SS497900,Cartesian,1776.310557,1808.0,http://194.83.155.90/olibcgi?infile=details.gl...,,12,Gower Commons / Tiroedd Comin Gwyr,GOWER,17.262288,97.180575,"{'rings': [[[251021.5, 190443.90000000037], [2...",Gower Commons/ Tiroedd Comin Gwyr,UK0012685,5,0,0.0,0.0
5,6,46,Gower Ash Woods / Coedydd Ynn Gwyr,UK0030157,Designated,n,2004-12-13,231.879424,0.0,257367.0,188205.0,-4.059710,51.574568,2500,OS GB GRID,1,NC,4.0,25,2003-02-28,CH/MG,"-4,3,35","51,34,28",SS573882,Cartesian,231.879424,2504.0,http://194.83.155.90/olibcgi?infile=details.gl...,,11,Gower Ash Woods / Coedydd Ynn Gwyr,GOWER,2.318445,99.984945,"{'rings': [[[257038.0700000003, 188061.25], [2...",Gower Ash Woods/ Coedydd Ynn Gwyr,UK0030157,2,0,0.0,0.0
6,7,49,Glannau Ynys Gybi / Holy Island Coast,UK0013046,Designated,p,2004-12-13,463.871478,0.0,220891.0,381723.0,-4.689391,53.302086,2500,OS GB GRID,1,,4.0,22,2007-09-03,BKS,"-4,41,22","53,18,8",SH208817,Cartesian,463.871478,1807.0,http://194.83.155.90/olibcgi?infile=details.gl...,,10,Glannau Ynys Gybi / Holy Island Coast,YNYS MON/ANGLESEY,4.482280,96.627630,"{'rings': [[[223615.13999999966, 383870.050000...",Glannau Ynys Gybi/ Holy Island Coast,UK0013046,3,0,0.0,0.0
7,8,50,Glannau Mon: Cors heli / Anglesey Coast: Saltm...,UK0020025,Designated,p,2004-12-13,1057.763512,0.0,238095.0,365501.0,-4.423231,53.161935,2500,OS GB GRID,1,,4.0,22,2002-05-07,BJ,"-4,25,

In [24]:
sac_aonb_final = sac_aonb_con_merge.groupby("AONB_NAME", as_index=False)["feat_count","fav_count"].sum()
sac_aonb_final['% Favourable'] = (sac_aonb_final['fav_count'] / sac_aonb_final['feat_count']*100)
sac_aonb_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,AONB_NAME,feat_count,fav_count,% Favourable
0,BRYNIAU CLWYD A DYFFRYN DYFRDWY/CLWYDIAN RANGE...,3,0.0,0.000000
1,GOWER,7,0.0,0.000000
2,LLYN,1,0.0,0.000000
3,WYE VALLEY (England),8,4.0,50.000000
4,WYE VALLEY (Wales),2,2.0,100.000000
5,YNYS MON/ANGLESEY,17,2.0,11.764706


### SAC Condition in National Parks ###

In [25]:
sac_np_50 = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SAC_NP_Intersect50"
sac_np = SpatialDataFrame.from_featureclass(sac_np_50)
sac_np.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,NP_NAME,AREA,PERCENTAGE,SHAPE
0,1,8,Usk Bat Sites / Safleodd Ystlumod Wysg,UK0014784,Designated,n,2004-12-13,1686.025494,0.0,319012.0,214582.0,-3.176541,51.823922,2500,OS GB GRID,1,NC,5.0,23,2002-04-30,WR,"-3,10,36","51,49,26",SO190145,Cartesian,1686.025494,1827.0,http://194.83.155.90/olibcgi?infile=details.gl...,,40,Usk Bat Sites / Safleodd Ystlumod Wysg,Brecon Beacons,16.860255,100.000000,"{'rings': [[[318621.4000000004, 216802.7699999..."
1,2,10,Sugar Loaf Woodlands,UK0030072,Designated,n,2004-12-13,174.163381,0.0,329521.0,216629.0,-3.024499,51.843753,2500,OS GB GRID,1,C,5.0,22,2007-09-03,BKS,"-3,1,28","51,50,38",SO295166,Cartesian,174.163381,2461.0,http://194.83.155.90/olibcgi?infile=details.gl...,,39,Sugar Loaf Woodlands,Brecon Beacons,1.741634,100.000000,"{'rings': [[[329986.0599999996, 216406.6999999..."
2,3,11,St David's / Ty Ddewi,UK0013045,Designated,p,2004-12-13,939.884184,0.0,172829.0,228522.0,-5.304216,51.908747,2500,OS GB GRID,1,NC,5.0,22,2002-08-06,"ABM / MG, BJ","-5,18,15","51,54,31",SM728285,Cartesian,939.884184,1825.0,http://194.83.155.90/olibcgi?infile=details.gl...,,38,St David's / Ty Ddewi,Pembrokeshire Coast,9.353696,99.519662,"{'rings': [[[177262.66000000015, 229700.140000..."
3,4,22,Rhinog,UK0012945,Designated,n,2004-12-13,3144.781854,0.0,264984.0,329739.0,-4.006366,52.848160,2500,OS GB GRID,1,NC,5.0,22,2002-09-05,MG,"-4,0,23","52,50,53",SH649297,Cartesian,3144.781854,1819.0,http://194.83.155.90/olibcgi?infile=details.gl...,,33,Rhinog,Snowdonia,31.447819,100.000000,"{'rings': [[[265000, 325053.25], [264397.5, 32..."
4,5,23,Preseli,UK0012598,Designated,n,2004-12-13,2704.368242,0.0,211066.0,232087.0,-4.751018,51.955055,2500,OS GB GRID,1,C,5.0,24,2007-09-03,BKS,"-4,45,4","51,57,18",SN110320,Cartesian,2704.368242,1818.0,http://194.83.155.90/olibcgi?infile=details.gl...,,32,Preseli,Pembrokeshire Coast,27.008693,99.870619,"{'rings': [[[216484.0599999996, 233223.6500000..."


In [26]:
sac_np_con_merge = pd.merge(sac_np,sac_feature_summary,left_on='Site_code', right_on='Site_Code_x')
sac_np_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,NP_NAME,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,1,8,Usk Bat Sites / Safleodd Ystlumod Wysg,UK0014784,Designated,n,2004-12-13,1686.025494,0.0,319012.000000,214582.000000,-3.176541,51.823922,2500,OS GB GRID,1,NC,5.0,23,2002-04-30,WR,"-3,10,36","51,49,26",SO190145,Cartesian,1686.025494,1827.0,http://194.83.155.90/olibcgi?infile=details.gl...,,40,Usk Bat Sites / Safleodd Ystlumod Wysg,Brecon Beacons,16.860255,100.000000,"{'rings': [[[318621.4000000004, 216802.7699999...",Usk Bat Sites/ Safleoedd Ystlumod Wysg,UK0014784,7,UK0014784,4.0,57.142857
1,2,10,Sugar Loaf Woodlands,UK0030072,Designated,n,2004-12-13,174.163381,0.0,329521.000000,216629.000000,-3.024499,51.843753,2500,OS GB GRID,1,C,5.0,22,2007-09-03,BKS,"-3,1,28","51,50,38",SO295166,Cartesian,174.163381,2461.0,http://194.83.155.90/olibcgi?infile=details.gl...,,39,Sugar Loaf Woodlands,Brecon Beacons,1.741634,100.000000,"{'rings': [[[329986.0599999996, 216406.6999999...",Sugar Loaf Woodlands,UK0030072,1,0,0.0,0.000000
2,3,11,St David's / Ty Ddewi,UK0013045,Designated,p,2004-12-13,939.884184,0.0,172829.000000,228522.000000,-5.304216,51.908747,2500,OS GB GRID,1,NC,5.0,22,2002-08-06,"ABM / MG, BJ","-5,18,15","51,54,31",SM728285,Cartesian,939.884184,1825.0,http://194.83.155.90/olibcgi?infile=details.gl...,,38,St David's / Ty Ddewi,Pembrokeshire Coast,9.353696,99.519662,"{'rings': [[[177262.66000000015, 229700.140000...",St David`s / Ty Ddewi,UK0013045,3,UK0013045,1.0,33.333333
3,4,22,Rhinog,UK0012945,Designated,n,2004-12-13,3144.781854,0.0,264984.000000,329739.000000,-4.006366,52.848160,2500,OS GB GRID,1,NC,5.0,22,2002-09-05,MG,"-4,0,23","52,50,53",SH649297,Cartesian,3144.781854,1819.0,http://194.83.155.90/olibcgi?infile=details.gl...,,33,Rhinog,Snowdonia,31.447819,100.000000,"{'rings': [[[265000, 325053.25], [264397.5, 32...",Rhinog,UK0012945,8,UK0012945,4.0,50.000000
4,5,23,Preseli,UK0012598,Designated,n,2004-12-13,2704.368242,0.0,211066.000000,232087.000000,-4.751018,51.955055,2500,OS GB GRID,1,C,5.0,24,2007-09-03,BKS,"-4,45,4","51,57,18",SN110320,Cartesian,2704.368242,1818.0,http://194.83.155.90/olibcgi?infile=details.gl...,,32,Preseli,Pembrokeshire Coast,27.008693,99.870619,"{'rings': [[[216484.0599999996, 233223.6500000...",Preseli,UK0012598,7,UK0012598,2.0,28.571429
5,6,26,Pembrokeshire Bat Sites and Bosherton Lakes / ...,UK0014793,Designated,n,2004-12-13,122.319940,0.0,196632.000000,195490.000000,-4.939325,51.621350,2500,OS GB GRID,1,,4.0,22,2004-07-30,BJ/MG,"-4,56,22","51,37,17",SR966954,Cartesian,122.319940,1815.0,http://194.83.155.90/olibcgi?infile=details.gl...,,29,Pembrokeshire Bat Sites and Bosherton Lakes / ...,Pembrokeshire Coast,1.222497,99.942537,"{'rings': [[[196216.90000000037, 195842.199999...",Pembrokeshire Bat Sites and Bosherston Lakes/ ...,UK0014793,4,UK0014793,2.0,50.000000
6,7,27,North West Pembrokeshire Commons / Comin Gogle...,UK0030229,Designated,n,2004-12-13,248.851894,0.0,177621.000000,227386.000000,-5.233936,51.900483,2500,OS GB GRID,1,NC,4.0,23,2002-02-05,MG/WR,"-5,14,2","51,54,2",SM776273,Cartesian,248.851894,2523.0,http://194.83.155.90/olibcgi?infile=details.gl...,,28,North West Pembrokeshire Commons / Comin Gogle...,Pembrokeshire Coast,2.488519,100.000000,"{'rings': [[[179243.5700000003, 228690.5800000...",North West Pembrokeshire Commons/ Comins Gogle...,UK0030229,5,0,0.0,0.000000
7,8,28,North Pembrokeshire Woodlands / Coedydd Gogled...,UK0030227,Designated,n,2004-12-13,315.314204,0.0,204607.000000,234529.000000,-4.846279,51.974756,2500,OS GB GRID,1,NC,5.0,30,2002-08-12,BJ,"-4,50,47","51,58,29",SN046345,Cartesian,315.314204,2522.0,http://194.83.155.90/olibcgi?infile=details.gl...,,27,North Pembrokeshire Woodlands / Coedydd Gogled...

In [27]:
sac_np_final = sac_np_con_merge.groupby("NP_NAME", as_index=False)["feat_count","fav_count"].sum()
sac_np_final['% Favourable'] = (sac_np_final['fav_count'] / sac_np_final['feat_count']*100)
sac_np_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,NP_NAME,feat_count,fav_count,% Favourable
0,Brecon Beacons,20,7.0,35.000000
1,Pembrokeshire Coast,38,11.0,28.947368
2,Snowdonia,73,21.0,28.767123


### SAC condition outside AONBs & National Parks ###

In [28]:
sac_out = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Wales_SAC_Out_AONB_NP"
sac_out = SpatialDataFrame.from_featureclass(sac_out)
sac_out.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE
0,1,2,Y Fenai a Bae Conwy / Menai Strait and Conwy Bay,UK0030202,Designated,y,2004-12-13,26501.735203,0.0,262947.0,372807.0,-4.054925,53.234548,2500,OS GB GRID,1,NC,6.0,32,2004-07-19,BKS MG,"-4,3,18","53,14,4",SH629728,Cartesian,26501.735203,2518.0,http://194.83.155.90/olibcgi?infile=details.gl...,,56.0,Y Fenai a Bae Conwy / Menai Strait and Conwy Bay,1.0,1.551633,0.585483,"{'rings': [[[274972.2999999998, 384110.5999999..."
1,2,7,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2004-12-13,141.389065,0.0,360563.0,204482.0,-2.572540,51.737600,2500,OS GB GRID,1,C,5.0,23,2002-03-01,WR,"-2,34,21","51,44,15",S0605044,Cartesian,141.389065,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,52.0,Wye Valley and Forest of Dean Bat Sites / Safl...,1.0,0.000045,0.003192,"{'rings': [[[360153.4749999996, 205186.9350000..."
2,3,14,River Wye / Afon Gwy (Wales),UK0012642,Designated,n,2001-03-16,1274.043071,0.0,297551.0,257865.0,-3.500796,52.209420,2500,OS GB GRID,1,NC,7.0,30,2005-07-18,EN/MG,"-3,30,3","52,12,34",SN975578,Cartesian,1274.043071,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,47.0,River Wye / Afon Gwy (Wales),1.0,1.830865,14.370512,"{'rings': [[[324446.4900000002, 245956.8900000..."
3,4,15,River Wye / Afon Gwy (England),UK0012642,Designated,n,2004-12-13,964.845245,0.0,356962.0,233596.0,-2.628330,51.999100,2500,OS GB GRID,1,NC,6.0,30,2003-06-17,BKS/CH/AN/MG,"-2,37,42","51,59,57",S0569335,Cartesian,964.845245,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,46.0,River Wye / Afon Gwy (England),1.0,4.753464,49.266598,"{'rings': [[[324402.2089999998, 247086.1870000..."
4,5,16,River Usk / Afon Wysg,UK0013007,Designated,p,2004-12-13,1014.524261,0.0,330190.0,211295.0,-3.013710,51.795900,2500,OS GB GRID,1,NC,5.0,30,2004-08-30,"BJ, SF, RhW, MG","-3,0,49","51,47,45",SO301112,Cartesian,1014.524261,1822.0,http://194.83.155.90/olibcgi?infile=details.gl...,,45.0,River Usk / Afon Wysg,1.0,4.040374,39.825309,"{'rings': [[[338944.95299999975, 198997.741000..."


In [35]:
# remove marine sacs
sac_out = sac_out.query('Marine=="n"', engine='python') 
sac_out

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE
1,2.0,7.0,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2004-12-13,141.389065,0.0,360563.0,204482.0,-2.572540,51.737600,2500.0,OS GB GRID,1.0,C,5.0,23.0,2002-03-01,WR,"-2,34,21","51,44,15",S0605044,Cartesian,141.389065,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,52.0,Wye Valley and Forest of Dean Bat Sites / Safl...,1.0,0.000045,0.003192,"{'rings': [[[360153.4749999996, 205186.9350000..."
2,3.0,14.0,River Wye / Afon Gwy (Wales),UK0012642,Designated,n,2001-03-16,1274.043071,0.0,297551.0,257865.0,-3.500796,52.209420,2500.0,OS GB GRID,1.0,NC,7.0,30.0,2005-07-18,EN/MG,"-3,30,3","52,12,34",SN975578,Cartesian,1274.043071,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,47.0,River Wye / Afon Gwy (Wales),1.0,1.830865,14.370512,"{'rings': [[[324446.4900000002, 245956.8900000..."
3,4.0,15.0,River Wye / Afon Gwy (England),UK0012642,Designated,n,2004-12-13,964.845245,0.0,356962.0,233596.0,-2.628330,51.999100,2500.0,OS GB GRID,1.0,NC,6.0,30.0,2003-06-17,BKS/CH/AN/MG,"-2,37,42","51,59,57",S0569335,Cartesian,964.845245,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,46.0,River Wye / Afon Gwy (England),1.0,4.753464,49.266598,"{'rings': [[[324402.2089999998, 247086.1870000..."
7,8.0,47.0,Glynllifon,UK0012661,Designated,n,2004-12-13,188.770611,0.0,245605.0,355096.0,-4.305999,53.070713,2500.0,OS GB GRID,1.0,NC,4.0,24.0,2003-11-01,"CH, MG, SDV","-4,18,22","53,4,15",SH456550,Cartesian,188.770611,1794.0,http://194.83.155.90/olibcgi?infile=details.gl...,,27.0,Glynllifon,1.0,0.032794,1.737242,"{'rings': [[[247104.1969999997, 356078.1030000..."
8,9.0,66.0,Corsydd Llyn / Lleyn Fens,UK0030187,Designated,n,2004-12-13,283.826147,0.0,231329.0,336605.0,-4.509219,52.900333,2500.0,OS GB GRID,1.0,NC,4.0,23.0,2002-04-15,PJ / MG,"-4,30,33","52,54,1",SH313366,Cartesian,283.826147,2517.0,http://194.83.155.90/olibcgi?infile=details.gl...,,21.0,Corsydd Llyn / Lleyn Fens,1.0,0.183772,6.474803,"{'rings': [[[229978.52099999972, 337461.471999..."
9,10.0,67.0,Corsydd Eifionydd / Eifionydd Fens,UK0030121,Designated,n,2004-12-13,144.540436,0.0,246055.0,347831.0,-4.295815,53.005581,2500.0,OS GB GRID,1.0,NC,25.0,23.0,2002-04-18,MG,"-4,17,45","53,0,20",SH460478,Cartesian,144.540436,2488.0,http://194.83.155.90/olibcgi?infile=details.gl...,,20.0,Corsydd Eifionydd / Eifionydd Fens,1.0,0.357633,24.742757,"{'rings': [[[246613.28000000026, 347531.619999..."
13,14.0,89.0,Blaen Cynon,UK0030092,Designated,n,2004-12-13,66.905182,0.0,294618.0,206631.0,-3.527953,51.748392,2500.0,OS GB GRID,1.0,C,4.0,23.0,2007-09-03,BKS,"-3,31,41","51,44,54",SN946066,Cartesian,66.905182,2469.0,http://194.83.155.90/olibcgi?infile=details.gl...,,8.0,Blaen Cynon,1.0,0.145454,21.740295,"{'rings': [[[294156.01999999955, 207003.289999..."
14,15.0,91.0,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,UK0012926,Designated,n,2004-12-13,27216.908451,0.0,291862.0,328022.0,-3.606871,52.838806,2500.0,OS GB GRID,1.0,NC,4.0,22.0,2002-08-08,BJ,"-3,36,25","52,50,20",SH918280,Cartesian,27216.908451,1788.0,http://194.83.155.90/olibcgi?infile=details.gl...,,7.0,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,1.0,115.963196,42.607042,"{'rings': [[[311259.3300000001, 333109.1820000..."
15,16.0,94.0,Afonydd Cleddau / Cleddau Rivers,UK0030074,Designated,n,2004-12-13,751.706479,0.0,193945.0,225023.0,-4.995603,51.885549,2500.0,OS GB GRID,1.0,NC,7.0,30.0,2003-08-28,"AN, MG,SF,RW,MG","-4,59,44","51,53,8",SM939250,Cartesian,751.706479,2462.0,http://194.83.155.90/olibcgi?infile=details.gl...,,4.0,Afonydd Cleddau / Cleddau Rivers,1.0,0.661126,8.795009,"{'rings': [[[186698, 230853.9810000006], [1869..."
18,19.0,3.0,Yerbeston Tops,UK0030305,Designated,n,2004-12-13,18.717423,0.0,

In [36]:
sac_out_con_merge = pd.merge(sac_out,sac_feature_summary,left_on='Site_code', right_on='Site_Code_x')
sac_out_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,2.0,7.0,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2004-12-13,141.389065,0.0,360563.0,204482.0,-2.572540,51.737600,2500.0,OS GB GRID,1.0,C,5.0,23.0,2002-03-01,WR,"-2,34,21","51,44,15",S0605044,Cartesian,141.389065,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,52.0,Wye Valley and Forest of Dean Bat Sites / Safl...,1.0,0.000045,0.003192,"{'rings': [[[360153.4749999996, 205186.9350000...",Wye Valley and Forest of Dean Bat Sites/ Safle...,UK0014794,2,UK0014794,2.0,100.000000
1,3.0,14.0,River Wye / Afon Gwy (Wales),UK0012642,Designated,n,2001-03-16,1274.043071,0.0,297551.0,257865.0,-3.500796,52.209420,2500.0,OS GB GRID,1.0,NC,7.0,30.0,2005-07-18,EN/MG,"-3,30,3","52,12,34",SN975578,Cartesian,1274.043071,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,47.0,River Wye / Afon Gwy (Wales),1.0,1.830865,14.370512,"{'rings': [[[324446.4900000002, 245956.8900000...",River Wye/ Afon Gwy,UK0012642,11,UK0012642,1.0,9.090909
2,4.0,15.0,River Wye / Afon Gwy (England),UK0012642,Designated,n,2004-12-13,964.845245,0.0,356962.0,233596.0,-2.628330,51.999100,2500.0,OS GB GRID,1.0,NC,6.0,30.0,2003-06-17,BKS/CH/AN/MG,"-2,37,42","51,59,57",S0569335,Cartesian,964.845245,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,46.0,River Wye / Afon Gwy (England),1.0,4.753464,49.266598,"{'rings': [[[324402.2089999998, 247086.1870000...",River Wye/ Afon Gwy,UK0012642,11,UK0012642,1.0,9.090909
3,8.0,47.0,Glynllifon,UK0012661,Designated,n,2004-12-13,188.770611,0.0,245605.0,355096.0,-4.305999,53.070713,2500.0,OS GB GRID,1.0,NC,4.0,24.0,2003-11-01,"CH, MG, SDV","-4,18,22","53,4,15",SH456550,Cartesian,188.770611,1794.0,http://194.83.155.90/olibcgi?infile=details.gl...,,27.0,Glynllifon,1.0,0.032794,1.737242,"{'rings': [[[247104.1969999997, 356078.1030000...",Glynllifon,UK0012661,1,0,0.0,0.000000
4,9.0,66.0,Corsydd Llyn / Lleyn Fens,UK0030187,Designated,n,2004-12-13,283.826147,0.0,231329.0,336605.0,-4.509219,52.900333,2500.0,OS GB GRID,1.0,NC,4.0,23.0,2002-04-15,PJ / MG,"-4,30,33","52,54,1",SH313366,Cartesian,283.826147,2517.0,http://194.83.155.90/olibcgi?infile=details.gl...,,21.0,Corsydd Llyn / Lleyn Fens,1.0,0.183772,6.474803,"{'rings': [[[229978.52099999972, 337461.471999...",Corsydd Llyn/ Lleyn Fens,UK0030187,4,UK0030187,1.0,25.000000
5,10.0,67.0,Corsydd Eifionydd / Eifionydd Fens,UK0030121,Designated,n,2004-12-13,144.540436,0.0,246055.0,347831.0,-4.295815,53.005581,2500.0,OS GB GRID,1.0,NC,25.0,23.0,2002-04-18,MG,"-4,17,45","53,0,20",SH460478,Cartesian,144.540436,2488.0,http://194.83.155.90/olibcgi?infile=details.gl...,,20.0,Corsydd Eifionydd / Eifionydd Fens,1.0,0.357633,24.742757,"{'rings': [[[246613.28000000026, 347531.619999...",Corsydd Eifionydd,UK0030121,3,UK0030121,1.0,33.333333
6,14.0,89.0,Blaen Cynon,UK0030092,Designated,n,2004-12-13,66.905182,0.0,294618.0,206631.0,-3.527953,51.748392,2500.0,OS GB GRID,1.0,C,4.0,23.0,2007-09-03,BKS,"-3,31,41","51,44,54",SN946066,Cartesian,66.905182,2469.0,http://194.83.155.90/olibcgi?infile=details.gl...,,8.0,Blaen Cynon,1.0,0.145454,21.740295,"{'rings': [[[294156.01999999955, 207003.289999...",Blaen Cynon,UK0030092,1,0,0.0,0.000000
7,15.0,91.0,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,UK0012926,Designated,n,2004-12-13,27216.908451,0.0,291862.0,328022.0,-3.606871,52.838806,2500.0,OS GB GRID,1.0,NC,4.0,22.0,2002-08-08,BJ,"-3,36,25","52,50,20",SH918280,Cartesian,27216.908451,1788.0,http://194.83.155.90/olibcgi?infile=details.gl...,,7.0,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,1.0,115.963196,42.607042,"{'rings': [[[311259.3300000001, 333109.1820000...",Berwyn a Mynyddoedd de Clwyd/ Berw

In [37]:
sac_out_final = sac_out_con_merge.groupby("SAC_name", as_index=False)["feat_count","fav_count"].sum()
sac_out_final['% Favourable'] = (sac_out_final['fav_count'] / sac_out_final['feat_count']*100)
sac_out_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,SAC_name,feat_count,fav_count,% Favourable
0,Aberbargoed Grasslands,2,0.0,0.000000
1,Afonydd Cleddau / Cleddau Rivers,8,1.0,12.500000
2,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,6,3.0,50.000000
3,Blackmill Woodlands,1,0.0,0.000000
4,Blaen Cynon,1,0.0,0.000000
5,Caeau Mynydd Mawr,2,0.0,0.000000
6,Cardiff Beech Woods,2,1.0,50.000000
7,Cernydd Carmel,5,2.0,40.000000
8,Coed Cwm Einion,1,0.0,0.000000
9,Coedwigoedd Dyffryn Elwy / Elwy Valley Woods,1,0.0,0.000000


In [38]:
Total_SAC_out_feat =sac_out_final["feat_count"].sum()
Total_SAC_out_fav =sac_out_final["fav_count"].sum()
Total_SAC_out = Total_SAC_out_fav/Total_SAC_out_feat*100
print(Total_SAC_out)

17.482517482517483


### SPAs outside of AONB and National parks ###

In [10]:
spa_out = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SPA_Out_AONB_NP_50_non_marine"
spa_out = SpatialDataFrame.from_featureclass(spa_out)
spa_out.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE
0,1,2,Berwyn,UK9013111,1998-01-16,24272.01,0.0,302278,329034,-3.606900,52.838799,2500,OS GB GRID,1,NC,4.0,2001-05-21,MG,"-3,36,25","52,50,20",SH918280,Spherical,0,24192.09,1840.0,302277.661354,329034.055748,http://194.83.155.90/olibcgi?infile=details.gl...,,3.0,Berwyn,1.0,66.626116,27.449776,"{'rings': [[[289437, 314718.5], [289500.5, 314..."
1,2,6,Dyfi Estuary / Aber Dyfi,UK9020284,2001-07-16,2056.51,0.0,265527,295947,-3.984444,52.544722,2500,OS GB GRID,1,NC,2.0,2000-11-21,SM,"-3,59,04","52,32,41",SN650956,Spherical,1,2049.44,2611.0,265000.575782,295627.053304,http://194.83.155.90/olibcgi?infile=details.gl...,,7.0,Dyfi Estuary / Aber Dyfi,1.0,5.349527,26.012710,"{'rings': [[[264091.1200000001, 293849], [2638..."
2,4,7,Elenydd - Mallaen,UK9014111,1996-01-12,30007.72,0.0,283631,263226,-3.706223,52.254860,2500,OS GB GRID,1,NC,4.0,2002-10-03,CH,"-3,42,22","52,15,17",SN821648,Spherical,0,29909.98,1844.0,283630.829794,263225.834075,http://194.83.155.90/olibcgi?infile=details.gl...,,NaN,None,NaN,NaN,NaN,"{'rings': [[[287155.8700000001, 255392.5899999..."


In [12]:
spa_out_con_merge = pd.merge(spa_out,spa_feature_summary,left_on='SPA_code', right_on='Site_Code_x')
spa_out_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SPA_Name,SPA_code,CLASS_DATE,CART_AREA,Riv_length,CentreX,CentreY,LONG,LAT,Map_Scale,PROJECTION,Vector,Code,Version,Edit_Date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,Off_Data,SPHER_AREA,ISIS_ID,Centre_X,Centre_Y,METADATA,GlobalID,OBJECTID_1,SPA_Name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count
0,1,2,Berwyn,UK9013111,1998-01-16,24272.01,0.0,302278,329034,-3.606900,52.838799,2500,OS GB GRID,1,NC,4.0,2001-05-21,MG,"-3,36,25","52,50,20",SH918280,Spherical,0,24192.09,1840.0,302277.661354,329034.055748,http://194.83.155.90/olibcgi?infile=details.gl...,,3.0,Berwyn,1.0,66.626116,27.449776,"{'rings': [[[289437, 314718.5], [289500.5, 314...",Berwyn,UK9013111,4,UK9013111,1.0
1,2,6,Dyfi Estuary / Aber Dyfi,UK9020284,2001-07-16,2056.51,0.0,265527,295947,-3.984444,52.544722,2500,OS GB GRID,1,NC,2.0,2000-11-21,SM,"-3,59,04","52,32,41",SN650956,Spherical,1,2049.44,2611.0,265000.575782,295627.053304,http://194.83.155.90/olibcgi?infile=details.gl...,,7.0,Dyfi Estuary / Aber Dyfi,1.0,5.349527,26.012710,"{'rings': [[[264091.1200000001, 293849], [2638...",Dyfi Estuary / Aber Dyfi,UK9020284,1,UK9020284,1.0
2,4,7,Elenydd - Mallaen,UK9014111,1996-01-12,30007.72,0.0,283631,263226,-3.706223,52.254860,2500,OS GB GRID,1,NC,4.0,2002-10-03,CH,"-3,42,22","52,15,17",SN821648,Spherical,0,29909.98,1844.0,283630.829794,263225.834075,http://194.83.155.90/olibcgi?infile=details.gl...,,NaN,None,NaN,NaN,NaN,"{'rings': [[[287155.8700000001, 255392.5899999...",Elenydd – Mallaen,UK9014111,2,UK9014111,2.0


In [14]:
spa_out_final = spa_out_con_merge.groupby("SPA_Name", as_index=False)["feat_count","fav_count"].sum()
spa_out_final['% Favourable'] = (spa_out_final['fav_count'] / spa_out_final['feat_count']*100)
spa_out_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,SPA_Name,feat_count,fav_count,% Favourable
0,Berwyn,4,1.0,25.0
1,Dyfi Estuary / Aber Dyfi,1,1.0,100.0
2,Elenydd - Mallaen,2,2.0,100.0


In [18]:
Total_SPA_out_feat = spa_out_final["feat_count"].sum()
Total_SPA_out_fav = spa_out_final["fav_count"].sum()
Total_SPA_out = Total_SPA_out_fav/Total_SPA_out_feat*100
print(Total_SPA_out)

57.14285714285714


## SAC's outside of AONBs only ##

In [9]:
# note marine removed in pro for this feature class
sac_out_aonb = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SAC_Out_AONB_50"
sac_out_aonb = SpatialDataFrame.from_featureclass(sac_out_aonb)
sac_out_aonb.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE
0,1,7,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2004-12-13,141.389065,0.0,360563.0,204482.0,-2.572540,51.737600,2500,OS GB GRID,1,C,5.0,23,2002-03-01,WR,"-2,34,21","51,44,15",S0605044,Cartesian,141.389065,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[360153.4749999996, 205186.9350000..."
1,2,14,River Wye / Afon Gwy (Wales),UK0012642,Designated,n,2001-03-16,1274.043071,0.0,297551.0,257865.0,-3.500796,52.209420,2500,OS GB GRID,1,NC,7.0,30,2005-07-18,EN/MG,"-3,30,3","52,12,34",SN975578,Cartesian,1274.043071,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[324446.4900000002, 245956.8900000..."
2,3,15,River Wye / Afon Gwy (England),UK0012642,Designated,n,2004-12-13,964.845245,0.0,356962.0,233596.0,-2.628330,51.999100,2500,OS GB GRID,1,NC,6.0,30,2003-06-17,BKS/CH/AN/MG,"-2,37,42","51,59,57",S0569335,Cartesian,964.845245,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[324402.2089999998, 247086.1870000..."
3,4,17,River Dee and Bala Lake / Afon Dyfrdwy a Llyn ...,UK0030252,Designated,n,2005-04-01,1149.884964,0.0,291051.0,340457.0,-3.623078,52.950382,2500,OS GB GRID,1,NC,3.0,25,2003-01-10,PJ/BKS/SL/BJ/SDV,"-3,37,23","52,57,1",SH910404,Cartesian,1149.884964,2525.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[342100.46999999974, 350098.480000..."
4,5,66,Corsydd Llyn / Lleyn Fens,UK0030187,Designated,n,2004-12-13,283.826147,0.0,231329.0,336605.0,-4.509219,52.900333,2500,OS GB GRID,1,NC,4.0,23,2002-04-15,PJ / MG,"-4,30,33","52,54,1",SH313366,Cartesian,283.826147,2517.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[229978.52099999972, 337461.471999..."


In [16]:
sac_out_aonb_con_merge = pd.merge(sac_out_aonb,sac_feature_summary,left_on='Site_code', right_on='Site_Code_x')
sac_out_aonb_con_merge

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_12_13,OBJECTID,SAC_name,Site_code,Status,Marine,NOTIF_DATE,Area_ha,Riv_length,CentreX,CentreY,LONG,LAT,MapScale,MapProject,Vector,Code,Version,Trancheno,Edit_date,User_id,LONG_DMS,LAT_DMS,NGR,REG_AREA,CART_AREA,ISIS_ID,METADATA,GlobalID,OBJECTID_1,SAC_name_1,OBJECTID_12,AREA,PERCENTAGE,SHAPE,Site_Name,Site_Code_x,feat_count,Site_Code_y,fav_count,% Favourable
0,1,7,Wye Valley and Forest of Dean Bat Sites / Safl...,UK0014794,Designated,n,2004-12-13,141.389065,0.0,360563.0,204482.0,-2.572540,51.737600,2500,OS GB GRID,1,C,5.0,23,2002-03-01,WR,"-2,34,21","51,44,15",S0605044,Cartesian,141.389065,1828.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[360153.4749999996, 205186.9350000...",Wye Valley and Forest of Dean Bat Sites/ Safle...,UK0014794,2,UK0014794,2.0,100.000000
1,2,14,River Wye / Afon Gwy (Wales),UK0012642,Designated,n,2001-03-16,1274.043071,0.0,297551.0,257865.0,-3.500796,52.209420,2500,OS GB GRID,1,NC,7.0,30,2005-07-18,EN/MG,"-3,30,3","52,12,34",SN975578,Cartesian,1274.043071,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[324446.4900000002, 245956.8900000...",River Wye/ Afon Gwy,UK0012642,11,UK0012642,1.0,9.090909
2,3,15,River Wye / Afon Gwy (England),UK0012642,Designated,n,2004-12-13,964.845245,0.0,356962.0,233596.0,-2.628330,51.999100,2500,OS GB GRID,1,NC,6.0,30,2003-06-17,BKS/CH/AN/MG,"-2,37,42","51,59,57",S0569335,Cartesian,964.845245,1823.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[324402.2089999998, 247086.1870000...",River Wye/ Afon Gwy,UK0012642,11,UK0012642,1.0,9.090909
3,4,17,River Dee and Bala Lake / Afon Dyfrdwy a Llyn ...,UK0030252,Designated,n,2005-04-01,1149.884964,0.0,291051.0,340457.0,-3.623078,52.950382,2500,OS GB GRID,1,NC,3.0,25,2003-01-10,PJ/BKS/SL/BJ/SDV,"-3,37,23","52,57,1",SH910404,Cartesian,1149.884964,2525.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[342100.46999999974, 350098.480000...",River Dee and Bala Lake/ Afon Dyfrdwy a Llyn T...,UK0030252,8,0,0.0,0.000000
4,5,66,Corsydd Llyn / Lleyn Fens,UK0030187,Designated,n,2004-12-13,283.826147,0.0,231329.0,336605.0,-4.509219,52.900333,2500,OS GB GRID,1,NC,4.0,23,2002-04-15,PJ / MG,"-4,30,33","52,54,1",SH313366,Cartesian,283.826147,2517.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[229978.52099999972, 337461.471999...",Corsydd Llyn/ Lleyn Fens,UK0030187,4,UK0030187,1.0,25.000000
5,6,91,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,UK0012926,Designated,n,2004-12-13,27216.908451,0.0,291862.0,328022.0,-3.606871,52.838806,2500,OS GB GRID,1,NC,4.0,22,2002-08-08,BJ,"-3,36,25","52,50,20",SH918280,Cartesian,27216.908451,1788.0,http://194.83.155.90/olibcgi?infile=details.gl...,,None,None,None,None,None,"{'rings': [[[311259.3300000001, 333109.1820000...",Berwyn a Mynyddoedd de Clwyd/ Berwyn and South...,UK0012926,6,UK0012926,3.0,50.000000


In [22]:
sac_out_aonb_final= sac_out_aonb_con_merge.groupby("SAC_name", as_index=False)["feat_count","fav_count"].sum()
sac_out_aonb_final['% Favourable'] = (sac_out_aonb_final['fav_count'] / sac_out_aonb_final['feat_count']*100)
sac_out_aonb_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,SAC_name,feat_count,fav_count,% Favourable
0,Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...,6,3.0,50.000000
1,Corsydd Llyn / Lleyn Fens,4,1.0,25.000000
2,River Dee and Bala Lake / Afon Dyfrdwy a Llyn ...,8,0.0,0.000000
3,River Wye / Afon Gwy (England),11,1.0,9.090909
4,River Wye / Afon Gwy (Wales),11,1.0,9.090909
5,Wye Valley and Forest of Dean Bat Sites / Safl...,2,2.0,100.000000


In [23]:
Total_SAC_out_aonb_feat =sac_out_aonb_final["feat_count"].sum()
Total_SAC_out_aonb_fav =sac_out_aonb_final["fav_count"].sum()
Total_SAC_out_aonb = Total_SAC_out_aonb_fav/Total_SAC_out_aonb_feat*100
print(Total_SAC_out_aonb)

19.047619047619047


Attempt at writing function to make outside calculations simpler

In [59]:
def FeaturesOutsideArea(AOI_dissolved, Features, join_field, intersect):  

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    Workspace = "S:\\DataUnit\\DptShare\\Consultancy\\Lodge\\Conservation and Policy\\DavidHampson\\Lost_Decade_PA_analysis\\ArcGIS Pro\\Protected_Area_Analysis_UK\\Protected_Area_Analysis_UK.gdb"

    # Process: Tabulate Intersection (Tabulate Intersection) (analysis)
    TabulateInters = "TabulateInters"
    arcpy.analysis.TabulateIntersection(in_zone_features=Features, zone_fields=join_field, in_class_features=AOI_dissolved, out_table=TabulateInters, class_fields=["OBJECTID"], sum_fields=[], xy_tolerance="", out_units="SQUARE_KILOMETERS")

    # Process: Add Join (Add Join) (management)
    Features_joined = arcpy.management.AddJoin(Features, in_field=join_field, join_table=TabulateInters, join_field=join_field, join_type="KEEP_ALL")[0]

    # uncomment the relevant line depending on SAC or SPA (SPA doesn't have marine field - remove marine before running)
    #Features_select = arcpy.management.SelectLayerByAttribute(Features_joined, selection_type="NEW_SELECTION", where_clause="TabulateInters.PERCENTAGE < " + intersect + " Or TabulateInters.PERCENTAGE IS NULL And (" + Features + ".Marine = 'n')", invert_where_clause="")
    Features_select = arcpy.management.SelectLayerByAttribute(Features_joined, selection_type="NEW_SELECTION", where_clause="TabulateInters.PERCENTAGE < " + intersect + " Or TabulateInters.PERCENTAGE IS NULL", invert_where_clause="")

    # Process: Feature Class to Feature Class (Feature Class to Feature Class) (conversion)
    Output = arcpy.conversion.FeatureClassToFeatureClass(in_features=Features_select, out_path=Workspace, out_name="Output", where_clause="")
    return Output

In [45]:
def average_condition_in_area (features,df_condition, features_join, condition_join, groupby):
    """Function to return average feature condition in area"""
    sdf_features = SpatialDataFrame.from_featureclass(features)
    features_con_merge = pd.merge(sdf_features,df_condition,left_on=features_join, right_on=condition_join)
    
    df_final= features_con_merge.groupby(groupby, as_index=False)["feat_count","fav_count"].sum()
    df_final['% Favourable'] = (df_final['fav_count'] / df_final['feat_count']*100)
    
    features_total =df_final["feat_count"].sum()
    fav_features_total =df_final["fav_count"].sum()
    fav_features_avg = fav_features_total/features_total*100
    
    return df_final, "features:" + str(features_total) + " fav features:" + str(fav_features_total) + " average:" +  str(fav_features_avg)
    

    

In [46]:
FeaturesOutsideArea("NRW_AONB_Polygon_Dissolve","NRW_SACPolygon","Site_code", "50")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Layer Name or Table View: Dataset NRW_SACPolygon does not exist or is not supported
Failed to execute (AddJoin).


In [41]:
#SACs outside AONBs
average_condition_in_area ("S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SAC_Out_AONB_50",sac_feature_summary, 'Site_code', 'Site_Code_x','SAC_name')

(                                             SAC_name  ...  % Favourable
0                              Aberbargoed Grasslands  ...      0.000000
1                   Afon Eden - Cors Goch Trawsfynydd  ...     20.000000
2                         Afon Gwyrfai a Llyn Cwellyn  ...     60.000000
3                    Afonydd Cleddau / Cleddau Rivers  ...     12.500000
4   Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...  ...     50.000000
5                                 Blackmill Woodlands  ...      0.000000
6                                         Blaen Cynon  ...      0.000000
7                                      Brecon Beacons  ...      0.000000
8                                        Cadair Idris  ...     38.461538
9                                   Caeau Mynydd Mawr  ...      0.000000
10                                Cardiff Beech Woods  ...     50.000000
11       Carmarthen Bay Dunes / Twyni Bae Caerfyrddin  ...     25.000000
12                                     Cernydd Car

In [42]:
#SACs outside NPs
average_condition_in_area ("S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SAC_Out_NP_50",sac_feature_summary, 'Site_code', 'Site_Code_x','SAC_name')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
__main__:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


(                                             SAC_name  ...  % Favourable
0                              Aberbargoed Grasslands  ...      0.000000
1                    Afonydd Cleddau / Cleddau Rivers  ...     12.500000
2        Alyn Valley Woods / Coedwigoedd Dyffryn Alun  ...      0.000000
3   Berwyn a Mynyddoedd De Clwyd / Berwyn and Sout...  ...     50.000000
4                                 Blackmill Woodlands  ...      0.000000
5                                         Blaen Cynon  ...      0.000000
6                                   Caeau Mynydd Mawr  ...      0.000000
7                                 Cardiff Beech Woods  ...     50.000000
8                                      Cernydd Carmel  ...     40.000000
9                                     Coed Cwm Einion  ...      0.000000
10       Coedwigoedd Dyffryn Elwy / Elwy Valley Woods  ...      0.000000
11  Coedwigoedd Penrhyn Creuddyn / Creuddyn Penins...  ...      0.000000
12                               Coedydd Llawr-y-g

In [50]:
FeaturesOutsideArea("NRW_AONB_Polygon_Dissolve","NRW_SPAPolygon_non_marine","SPA_code", "50")

<Result 'S:\\DataUnit\\DptShare\\Consultancy\\Lodge\\Conservation and Policy\\DavidHampson\\Lost_Decade_PA_analysis\\ArcGIS Pro\\Protected_Area_Analysis_UK\\Protected_Area_Analysis_UK.gdb\\Output'>

In [57]:
#SPAs outside AONBs
average_condition_in_area ("S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SPA_Out_AONB_50",spa_feature_summary, 'SPA_code', 'Site_Code_x','SPA_Name')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
__main__:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


(                                SPA_Name  feat_count  fav_count  % Favourable
0                                 Berwyn           4        1.0     25.000000
1                     Castlemartin Coast           1        1.0    100.000000
2          Craig yr Aderyn (Bird's Rock)           2        0.0      0.000000
3               Dyfi Estuary / Aber Dyfi           1        1.0    100.000000
4                      Elenydd - Mallaen           2        2.0    100.000000
5                Migneint-Arenig-Dduallt           3        1.0     33.333333
6  Ramsey and St David's Peninsula Coast           1        1.0    100.000000, 'features:14 fav features:7.0 average:50.0')

In [58]:
FeaturesOutsideArea("NRW_NATIONAL_PARKPolygon_Dissolve","NRW_SPAPolygon_non_marine","SPA_code", "50")

<Result 'S:\\DataUnit\\DptShare\\Consultancy\\Lodge\\Conservation and Policy\\DavidHampson\\Lost_Decade_PA_analysis\\ArcGIS Pro\\Protected_Area_Analysis_UK\\Protected_Area_Analysis_UK.gdb\\Output'>

In [60]:
#SPAs outside NPs
average_condition_in_area ("S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\WALES_SPA_Out_NP_50",spa_feature_summary, 'SPA_code', 'Site_Code_x','SPA_Name')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
__main__:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


(                                            SPA_Name  ...  % Favourable
0                                             Berwyn  ...          25.0
1                           Dyfi Estuary / Aber Dyfi  ...         100.0
2                                  Elenydd - Mallaen  ...         100.0
3              Glannau Ynys Gybi / Holy Island Coast  ...          50.0
4  Mynydd Cilan, Trwyn y Wylfa ac Ynysoedd Sant T...  ...          50.0

[5 rows x 4 columns], 'features:11 fav features:6.0 average:54.54545454545454')